# Imports

In [1]:
import pandas as pd

# Defining a handy function

In [2]:
def TransformColumns(df):
    """Replaces the spaces with underlines and converts to lowecase the column names of a given DataFrame `df`.
    
    Parameters
    ----------
    df : DataFrame with messy column names
    
    Returns
    -------
    None
    
    Raises
    ------
    AssertionError
        * If `df` is not a ``pandas.DataFrame``.
    """
    
    try:
        assert type(df) is pd.core.frame.DataFrame, "Invalid input. Must be a DataFrame."
        df.columns = [c.lower().replace(' ', '_') for c in df.columns]
        return None
    except AssertionError as msg:
        print(msg)
        return None

# Implementing solution

### Declaring variables

In [3]:
dictMeses = {1: 'jan', 2:'fev', 3:'mar', 4:'abr', 5:'mai', 6:'jun', 7:'jul', 8:'ago', 9:'set', 10:'out', 11:'nov', 12:'dez'}

### Importing and preparing DataFrames

Here all DataFrames used in this solution are imported and transformed with the previously declared function TransformColumns. They are namely: <br>
- dfGroupAnalyst: A list of analysts to which each product group is assigned;
- dfProd: A product log, with basic infos about all the registered products;
- dfSales: The sales from last twelve months for each product;
- dfForecast1 - dfForecast8: The sales forecasted by the analysts from january/2019 to july/2019, separated in 7 different Dataframes
- dfDisregard: A list of products that should be disregarded from the calculation due to business decisions.

In [4]:
dfGroupAnalyst = pd.read_excel('./Inputs/0_Group per analyst.xls')
dfProd = pd.read_excel('./Inputs/1_Products.xls', sheet_name='Plan1', usecols='A, C:D')
dfSales = pd.read_excel('./Inputs/2_SALES.xlsx')
dfForecast1 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[1])
dfForecast2 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[2])
dfForecast3 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[3])
dfForecast4 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[4])
dfForecast5 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[5])
dfForecast6 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[6])
dfForecast7 = pd.read_excel('./Inputs/3_FORECASTS.xlsx', sheet_name=dictMeses[7])
dfDisregard = pd.read_excel('./Inputs/4_DISREGARD.xlsx', sheet_name='Disregard')

TransformColumns(dfGroupAnalyst)
TransformColumns(dfProd)
TransformColumns(dfSales)

From our Sales Dataframe (dfSales), let's filter out all products sold that have been discontinued and that are unavailabe with no reorder, as well as all products where 'griffe' is different than brands B and C.

In [5]:
dfSales = dfSales.loc[dfSales.griffe != 'Brand B']
dfSales = dfSales.loc[dfSales.griffe != 'Brand C']

dfSales = dfSales.loc[dfSales.linha != 'DISCONTINUED']
dfSales = dfSales.loc[dfSales.linha != 'UNAVAILABLE - NO REORDER']

### Handling the Dataframes and calculating the MAPE

Let's now merge the dfProd, dfGroupAnalyst and dfSales Dataframes in order to have all the infos in one place.

In [6]:
dfMAPE = dfProd.merge(dfGroupAnalyst, on='grupo_produto', how='left')
dfMAPE = dfSales.merge(dfMAPE, left_on='produto',  right_on='produto', how='left')

In order to merge the infos presented at the 7 forecast DataFrames, let's first iterate through them and append the infos to an auxiliary DataFrame (dfAux). Then, we will merge dfAux to dfMAPE in order to achieve the wanted DataFrame.

In [8]:
dfAux = pd.DataFrame()

ForecastList = [dfForecast1, dfForecast2, dfForecast3, dfForecast4, dfForecast5, dfForecast6, dfForecast7]

for x in range(0,7):
    dfWorked = ForecastList[x]
    dfWorked = dfWorked.melt(id_vars="produto", value_vars=list(dfWorked.columns)[-1], var_name='mes', value_name='dfu')
    dfWorked.mes = dfWorked.mes.astype(int)
    dfAux = dfAux.append(dfWorked)

dfMAPE = dfMAPE.merge(dfAux, left_on=['produto', 'mes'], right_on=['produto', 'mes'], how='left')

To complete the infos, let´s disregard the procuts in the sales months presented on dfDisregard, and then calculate the MAPE for every product in every month.

In [9]:
dfDisregard['check'] = 1
dfMAPE = dfMAPE.merge(dfDisregard, left_on=['produto', 'mes'], right_on=['produto', 'mes'], how='left')
dfMAPE = dfMAPE.loc[dfMAPE.check != 1]

cols = ['produto',  'desc_produto', 'grupo_produto', 'subgrupo_produto', 'analista', 'linha', 'mes', 'dfu', 
        'qtde_vendida', 'qtde_x_preco',  'mape']

dfMAPE['mape'] = ((dfMAPE['qtde_vendida'] - dfMAPE['dfu'])/dfMAPE['qtde_vendida'])
dfMAPE = dfMAPE[cols]

### Calculating WMAPE

In order to measure the impact of each prediction error (WMAPE), let's add 3 columns to dfMAPE:
- Percentage per month
- Percentage per month and analyst
- Percentage per month and group

After that, the Weighed MAPE can be calculated for each case.

Adding columns for WMAPE per month:

In [10]:
df = dfMAPE[['mes', 'qtde_x_preco']].groupby(by='mes').sum()
df.reset_index(inplace=True)
df.rename(columns={'qtde_x_preco': 'total'}, inplace=True)

dfMAPE = dfMAPE.merge(df, left_on='mes', right_on='mes', how='left')
dfMAPE['part_mes'] = dfMAPE['qtde_x_preco'] / dfMAPE['total']
dfMAPE.drop(columns='total', inplace=True)

dfMAPE['wmape_mes'] = abs(dfMAPE['mape']) * dfMAPE['part_mes']

Adding columns for WMAPE per month and analyst:

In [11]:
df = dfMAPE[['mes', 'analista', 'qtde_x_preco']].groupby(by=['mes', 'analista']).sum()
df.reset_index(inplace=True)
df.rename(columns={'qtde_x_preco': 'total'}, inplace=True)

dfMAPE = dfMAPE.merge(df, left_on=['mes', 'analista'], right_on=['mes', 'analista'], how='left')
dfMAPE['part_mes_analista'] = dfMAPE['qtde_x_preco'] / dfMAPE['total']
dfMAPE.drop(columns='total', inplace=True)

dfMAPE['wmape_mes_analista'] = abs(dfMAPE['mape']) * dfMAPE['part_mes_analista']

Adding columns for WMAPE per month and group:

In [12]:
df = dfMAPE[['mes', 'grupo_produto', 'qtde_x_preco']].groupby(by=['mes', 'grupo_produto']).sum()
df.reset_index(inplace=True)
df.rename(columns={'qtde_x_preco': 'total'}, inplace=True)

dfMAPE = dfMAPE.merge(df, left_on=['mes', 'grupo_produto'], right_on=['mes', 'grupo_produto'], how='left')
dfMAPE['part_mes_grupo_produto'] = dfMAPE['qtde_x_preco'] / dfMAPE['total']
dfMAPE.drop(columns='total', inplace=True)

dfMAPE['wmape_mes_grupo_produto'] = abs(dfMAPE['mape']) * dfMAPE['part_mes_grupo_produto']

# Creating the final DataFrames and exporting the files

In [15]:
writer = pd.ExcelWriter('WMAPE Calculado.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
dfMAPE.to_excel(writer, sheet_name='Base', index=False)

#WMAPE per month
df = dfMAPE[['mes', 'wmape_mes']].groupby(by=['mes']).sum()
df.to_excel(writer, sheet_name='WMAPE mensal total')

#WMAPE per month and analyst
df = dfMAPE[['mes', 'analista', 'wmape_mes_analista']].groupby(by=['mes', 'analista']).sum()
df.to_excel(writer, sheet_name='WMAPE mensal-analista')

#WMAPE per month and group
df = dfMAPE[['mes', 'grupo_produto', 'wmape_mes_grupo_produto']].groupby(by=['mes', 'grupo_produto']).sum()
df.to_excel(writer, sheet_name='WMAPE mensal-grupo')

#Evolution of forecast accuracy for the first analyst
df = dfMAPE.loc[dfMAPE.analista == "Analyst #1"].pivot_table(values="wmape_mes_analista", index="grupo_produto", 
                                                       columns="mes", aggfunc='sum')
df.to_excel(writer, sheet_name='Evolution Analyst #1')

#Evolution of forecast accuracy for the second analyst
df = dfMAPE.loc[dfMAPE.analista == "Analyst #2"].pivot_table(values="wmape_mes_analista", index="grupo_produto", 
                                                      columns="mes", aggfunc='sum')
df.to_excel(writer, sheet_name='Evolution Analyst #2')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

FileCreateError: [Errno 13] Permission denied: 'WMAPE Calculado.xlsx'